In [1]:
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
from tqdm import tqdm

import load_data
import tokenize_script
import dataset
import model

In [2]:
DATA_PATH = '../script_files/joined/raw.txt'

In [3]:
dialogue, screen_directions = load_data.get_dialogue_and_stage_directions(DATA_PATH)
tokenizer = tokenize_script.Tokenizer()
tokenizer.fit_on_texts(dialogue + screen_directions)

In [5]:
dialogue, screen_directions = load_data.get_three_line_dialogue_and_stage_directions(DATA_PATH)

In [6]:
tokenized_dialogue = tokenizer.texts_to_sequences(dialogue)
print("done")
tokenized_screen_directions = tokenizer.texts_to_sequences(screen_directions)

done


In [ ]:
labeled_data = []
for line in tokenized_dialogue:
    labeled_data.append((line, 0))
for line in tokenized_screen_directions:
    labeled_data.append((line, 1))

In [6]:
script_dataset = dataset.ScriptDataset(labeled_data)
train_dataset, test_dataset = torch.utils.data.random_split(script_dataset, [0.7, 0.3])

In [16]:
BATCH_SIZE = 64

In [8]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = BATCH_SIZE, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = BATCH_SIZE, shuffle = True)

In [20]:
BLOCKS = 3
FILTERS = 32
EMBEDDING_DIM = 200
KERNEL_SIZE = 3
DROPOUT = 0.3
LEARNING_RATE = 1E-4
NUM_EPOCHS = 10

device = 'mps'

In [21]:
classifier_model = model.Classifier(blocks=BLOCKS, filters=FILTERS, kernel_size=KERNEL_SIZE,
                                    embedding_dim=EMBEDDING_DIM, dropout_rate=DROPOUT, pool_size=KERNEL_SIZE, 
                                    input_shape=[BATCH_SIZE, len(train_dataset[0][0])], num_features=tokenizer.get_vocab_length()).to(device)

In [15]:
criterion = nn.BCELoss()
sigmoid = nn.Sigmoid()
optimizer = torch.optim.Adam(classifier_model.parameters(), lr=LEARNING_RATE)

In [51]:
for epoch in range(NUM_EPOCHS):
    total_train_loss = 0
    classifier_model.train()
    for x, y in tqdm(train_loader):
        x = torch.stack(x, dim = 1)
        x = x.to(device=device)
        x = torch.tensor(x)
        y = y.to(device)

        optimizer.zero_grad()
        outputs = classifier_model(x)
        print(outputs.shape) 
        loss = criterion(sigmoid(outputs), y)
        loss.backward()
        optimizer.step()
    print(f'loss: {total_train_loss/(epoch + 1)}')

  0%|                                                                                                              | 0/32142 [00:00<?, ?it/s]/var/folders/wd/3g59h2m12pn7ykcslq8mj8s40000gn/T/ipykernel_1591/1302011949.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x)
  0%|                                                                                                              | 0/32142 [00:00<?, ?it/s]

torch.Size([64, 200, 144])


ValueError: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 200, 144])) is deprecated. Please ensure they have the same size.